In [10]:
import requests
from bs4 import BeautifulSoup

import jieba
from zhon import hanzi as zh_hanzi

from cedict_utils.cedict import CedictParser

from dragonmapper import transcriptions
from dragonmapper import hanzi

import urllib.request

import numpy as np
import io

from PIL import Image

from pathlib import Path

import hashlib

from sys import platform

import datetime
import math

from collections import defaultdict

from gensim.parsing.preprocessing import remove_stopwords
import string

import spacy
nlp = spacy.load("en_core_web_sm")

In [2]:
class Sentence():
    
    def __init__(self, sentence, seg_list, english):
        self.sentence = sentence 
        self.seg_list = seg_list
        self.english = english
    
    def __str__(self):
        return " ".join(self.seg_list) + "\n" + self.english
    
    def score(self, level=3) -> int:
        level_dict = dict()
        score = 0
        for w in self.seg_list:
            w_level = hsk_level_new(w)
            level_dict[w] = w_level
            if w_level in ['4', '5']:
                score += 1
            elif w_level in ['6', '7-9']:
                score += 2
            elif w_level == "0":
                score += 3
        return score

In [3]:
PUNCTUATION = set(zh_hanzi.punctuation)

parser = CedictParser()
entries = parser.parse()

CEDICT = dict()
for e in entries:
    CEDICT[e.simplified] = e

In [4]:
HSK = dict()
words = set()
resource_path = "/home/daniel/programming/chinese/chinese-tools/Mandarin corner/resources/hsk3.0-"
levels = [str(i) for i in list(range(1,7))] + ["7-9"]

for level in levels:
    file_path = resource_path + level + '.txt'
    with open(file_path) as f:
        text = f.read()
        text_words = text.split("\n")
        text_words = set(text_words)
        words.update(text_words)
        HSK[level] = words.copy()

In [5]:
def hsk_level_new(word: str) -> str:

    for i in range(1,7):
        if word in HSK[str(i)]:
            return str(i)

    if word in HSK["7-9"]:
        return "7-9"
    
    return "0"

In [6]:
WORD = "酸痛"
# WORD = "还清"
url = "https://www.purpleculture.net/sample-sentences/?word=" + WORD
url

'https://www.purpleculture.net/sample-sentences/?word=酸痛'

In [7]:
response = requests.get(url)

if response.status_code != 200:
    print("ERROR")

In [8]:
soup = BeautifulSoup(response.text, 'html.parser')

In [9]:
Sentences = list()

sample_sentences = soup.findAll('span', attrs={'class':'sc samplesen'})
print("sentences:", len(sample_sentences))
en_sentences = soup.findAll('div', attrs={'class':'sample_en'})

seg_map = dict()

for i, sent in enumerate(sample_sentences):
    chars = sent.findAll('span', attrs={'class':'cnchar'})
    char_sent = "".join(char.text for char in chars)
#     print(char_sent, en_sentences[i].text)
    
    seg_list = list(jieba.cut(char_sent, cut_all=False))
#     pinyin = " ".join(hanzi.to_pinyin(word) for word in seg_list)
    
    seg_map[char_sent] = seg_list
#     print(seg_map)
#     print(char_sent, seg_list, en_sentences[i].text)
    Sentences.append(Sentence(char_sent, seg_list, en_sentences[i].text))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


sentences: 20


Loading model cost 0.403 seconds.
Prefix dict has been built successfully.


In [23]:
for i in Sentences:
    print(i.sentence)

看着六岁的儿子在花园扭断那些杜鹃花枝，我的心一下子酸痛起来，这一天就这么开始了。
以前他什么也不怕，现在他会找安闲自在：刮风下雨，他都不出车；身上有点酸痛，也一歇就是两三天。
请别让我们的眼睛因渴望见到你的面容而酸痛。
我的帽子已经歪斜，我的围裙已经脏了，我的脚酸痛。
那么研究人员如何解释有些人穿上塑身鞋后出现酸痛或者明显反应的情况呢？
如果你每30，40分钟站立休息一会或者舒展一下双脚，这样可以减少肌肉疲劳而产生的酸痛感。
不得不承认，我有时很喜欢腿部那种经历高强度锻炼或比赛所引发的肌肉酸痛感。
在一段艰苦的体育锻炼或是一段长途跑之后，你也许感觉到全身的肌肉酸痛。
对他们来说，这是对死去的两代人的酸痛记忆。
与老年人相比，儿童可较经常出现发热、酸痛和头痛。
运动过度的另一个指标是肌肉酸痛持续数天不消失。
如果在健身之后你太早吃东西的话，你可能最小化肌肉僵硬和酸痛。
许多专业跑步者会洗冰浴以减轻跑步后的酸痛。如果你不能忍受冰浴，在酸痛处敷上冰袋。
这是因为通过降低运动强度能减少肌肉中导致酸痛和其他不适感觉的乳酸的缘故。
任何人一旦出现突然发烧、咳嗽和肌肉酸痛等流感症状，应避免去工作场所或公共交通设施，赶紧去求医。
在冬天很难到达该地，但在夏天，当道路上没有什么车辆时，徒步旅行者会在到来后在地的热水中快速浸泡，以缓和他们肌肉的酸痛。这些地表水最后汇聚成细流小溪。
头晕、失眠、全身酸痛、口干舌燥……长期使用电脑的人都会遇到这些情况。
可是你来我这里抱怨你的膝盖疼，你的背痛的要死，你的脚也酸痛，你还没上了半截楼梯你就喘不上气来了。
只拿那些你感兴趣的公司的宣传品，因为你会阅读它们，否则背着一书包各公司的宣传品走来走去会让你胳臂酸痛。
叫女佣人拿去给你做茶点：区区一物，你不必客气，因为它又软又滑，喉咙酸痛吃下去也不碍事。


In [15]:
Sentences.sort(key=Sentence.score)

In [11]:
soup.findAll('div', attrs={'class' : 'samplesen'})

[]

In [16]:
print(Sentences[0].sentence)
print(Sentences[-1].sentence)

那么研究人员如何解释有些人穿上塑身鞋后出现酸痛或者明显反应的情况呢？
在冬天很难到达该地，但在夏天，当道路上没有什么车辆时，徒步旅行者会在到来后在地的热水中快速浸泡，以缓和他们肌肉的酸痛。这些地表水最后汇聚成细流小溪。


In [59]:
url2 = "https://www.purpleculture.net/dictionary-details/?word=还清"
response = requests.get(url2)
soup = BeautifulSoup(response.text, 'html.parser')
en_sentences = soup.findAll('div', attrs={'class':'sample_en'})

5

In [28]:
en_sentences

[<div class="sample_en" id="ensen1">To be honest, I seldom receive others' favor, but this time I really owe you a lot and do not know when I can pay you back. </div>,
 <div class="sample_en" id="ensen2">Pay off your credit cards every month. </div>,
 <div class="sample_en" id="ensen3">For example, you need to know about one another' s saving and spending habits like how often (if ever) you balance your checkbooks, pay your credit lines, and research big purchases before you buy. </div>,
 <div class="sample_en" id="ensen4">If all of these are covered and you still have cash left over (which you will, given some time), the next step is to pay off all of your debts. </div>,
 <div class="sample_en" id="ensen5">Think about what could change in your life if you paid off that credit card? </div>]

In [1]:
import OpenHowNet
hownet_dict = OpenHowNet.HowNetDict()

Important data file "resources/sememe_all" lost, please run `OpenHowNet.download()`.


In [2]:
OpenHowNet.download()

resources/resources.zip: 72948KB [00:17, 4246.46KB/s]                                                             


In [3]:
result_list = hownet_dict.get_sense("苹果")

AttributeError: 'HowNetDict' object has no attribute 'en_map'

In [12]:
sentence = "So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her."
doc = nlp(sentence)
nlp_sentence = list(doc.sents)[0]
nlp_list_sent = []
for word in nlp_sentence:
    nlp_list_sent.append(word.lemma_)
# print(nlp_list_sent)

filtered_sent = " ".join(nlp_list_sent)
filtered_sent = remove_stopwords(sentence)
filtered_sent = filtered_sent.translate(str.maketrans('', '', string.punctuation))
filtered_sent = filtered_sent.split(' ')
filtered_sent = [i.lower() for i in filtered_sent]
filtered_sent = set(filtered_sent)
# for word in filtered_sent:
#     lemma_sent.append()
print(filtered_sent)

{'sleepy', 'worth', 'daisies', 'close', 'stupid', 'her', 'so', 'could', 'making', 'white', 'day', 'rabbit', 'eyes', 'considering', 'feel', 'trouble', 'hot', 'pleasure', 'ran', 'pink', 'daisychain', 'as', 'picking', 'getting', 'suddenly', 'mind'}


In [13]:
CEDICT["酸痛"].meanings

['to ache']

In [28]:
for word in nlp_sentence:
    if word.pos_ == "VERB":
        print(word, word.lemma_)

considering consider
made make
feel feel
making make
getting get
picking pick
ran run


In [41]:
print(sentence)
print(remove_stopwords(sentence))

So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.
So considering mind (as could, hot day feel sleepy stupid), pleasure making daisy-chain worth trouble getting picking daisies, suddenly White Rabbit pink eyes ran close her.


In [35]:
sent_list

['So',
 'she',
 'was',
 'considering',
 'in',
 'her',
 'own',
 'mind',
 '(as',
 'well',
 'as',
 'she',
 'could,',
 'for',
 'the',
 'hot',
 'day',
 'made',
 'her',
 'feel',
 'very',
 'sleepy',
 'and',
 'stupid),',
 'whether',
 'the',
 'pleasure',
 'of',
 'making',
 'a',
 'daisy-chain',
 'would',
 'be',
 'worth',
 'the',
 'trouble',
 'of',
 'getting',
 'up',
 'and',
 'picking',
 'the',
 'daisies,',
 'when',
 'suddenly',
 'a',
 'White',
 'Rabbit',
 'with',
 'pink',
 'eyes',
 'ran',
 'close',
 'by',
 'her.']